# DQN for Atari Pong

Clean implementation organized into separate cells.

**Run cells in order:**
1. Imports
2. Wrappers
3. Model
4. Replay Buffer
5. DQN Agent
6. Training

In [1]:
# Cell 1: Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import cv2
import gymnasium as gym
import ale_py  # Registers ALE environments
from collections import deque, namedtuple
from tqdm import tqdm
import random
import os

print("✅ Imports complete")

✅ Imports complete


In [2]:
# Cell 2: Wrappers for Atari preprocessing
# Based on OpenAI Baselines but adapted for gymnasium

class NoopResetEnv(gym.Wrapper):
    """Sample initial states by taking random number of no-ops on reset."""
    def __init__(self, env, noop_max=30):
        super().__init__(env)
        self.noop_max = noop_max
        self.noop_action = 0
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        if self.noop_max > 0:
            noops = np.random.randint(1, self.noop_max + 1)
            for _ in range(noops):
                obs, _, terminated, truncated, _ = self.env.step(self.noop_action)
                if terminated or truncated:
                    obs, info = self.env.reset(**kwargs)
        return obs, info


class MaxAndSkipEnv(gym.Wrapper):
    """Return only every `skip`-th frame, max-pooling across last 2 frames."""
    def __init__(self, env, skip=4):
        super().__init__(env)
        self._skip = skip
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=np.uint8)

    def step(self, action):
        total_reward = 0.0
        terminated = False
        truncated = False
        info = {}

        for i in range(self._skip):
            obs, reward, term, trunc, inf = self.env.step(action)

            # Always write the most recent observations; avoids stale frames if episode ends early
            self._obs_buffer[i % 2] = obs

            total_reward += reward
            terminated = terminated or term
            truncated = truncated or trunc
            info = inf

            if terminated or truncated:
                break

        max_frame = self._obs_buffer.max(axis=0)
        return max_frame, total_reward, terminated, truncated, info


class WarpFrame(gym.ObservationWrapper):
    """Warp frames to 84x84 grayscale as done in the Nature paper."""
    def __init__(self, env):
        super().__init__(env)
        self.width = 84
        self.height = 84
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(self.height, self.width), dtype=np.uint8
        )
    
    def observation(self, frame):
        if len(frame.shape) == 3:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame


class ClipRewardEnv(gym.RewardWrapper):
    """Bin reward to {+1, 0, -1} by its sign."""
    def reward(self, reward):
        return np.sign(reward)


class FrameStack(gym.Wrapper):
    """Stack k last frames. Returns lazy array, which is much more memory efficient."""
    def __init__(self, env, k=4):
        super().__init__(env)
        self.k = k
        self.frames = deque([], maxlen=k)
        obs_shape = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(k, *obs_shape), dtype=np.uint8
        )
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        for _ in range(self.k):
            self.frames.append(obs)
        return self._get_obs(), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, terminated, truncated, info
    
    def _get_obs(self):
        assert len(self.frames) == self.k
        return np.stack(self.frames, axis=0)

class FireResetEnv(gym.Wrapper):
    """Press FIRE button at the start of each episode to begin gameplay."""
    def __init__(self, env):
        super().__init__(env)
        # Action 1 is typically FIRE in Atari games
        # For Pong, we need to press FIRE to start the game
        self.fire_action = 1
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        # Press FIRE to start the game
        obs, _, terminated, truncated, info = self.env.step(self.fire_action)
        if terminated or truncated:
            # If game ended immediately, reset again
            obs, info = self.env.reset(**kwargs)
        return obs, info


def make_atari_env(env_id='ALE/Pong-v5', max_episode_steps=108000, sticky_actions=False):
    """
    Create and wrap Atari environment.

    - Use ALE/Pong-v5 but force frameskip=1.
    - Then MaxAndSkipEnv(skip=4) becomes the ONLY frameskip.
    """
    env = gym.make(
        env_id,
        frameskip=1,  # critical: prevents 4x4=16 frame skipping
        repeat_action_probability=0.0,
        full_action_space=False,  # default is already reduced actions; keeping explicit
    )

    # (optional) keep your episode limit hack
    env._max_episode_steps = max_episode_steps

    env = NoopResetEnv(env, noop_max=30)
    env = FireResetEnv(env)  # ADD THIS - press FIRE to start game
    env = MaxAndSkipEnv(env, skip=4)
    env = WarpFrame(env)
    env = ClipRewardEnv(env)
    env = FrameStack(env, k=4)
    return env



# Test the wrapper
test_env = make_atari_env()
test_obs, _ = test_env.reset()
print(f"✅ Wrappers working! Observation shape: {test_obs.shape}, dtype: {test_obs.dtype}")
print(f"   Action space: {test_env.action_space}")
test_env.close()

✅ Wrappers working! Observation shape: (4, 84, 84), dtype: uint8
   Action space: Discrete(6)


In [3]:
# Cell 3: DQN Network Model

class DQNNetwork(nn.Module):
    """DQN CNN network for Atari games."""
    def __init__(self, n_actions, n_frames=4):
        super(DQNNetwork, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(n_frames, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Fully connected layers
        # Output size after conv layers: 7x7x64 = 3136
        self.fc1 = nn.Linear(7 * 7 * 64, 512)
        self.fc2 = nn.Linear(512, n_actions)
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Normalize to [0, 1]
        x = x.float() / 255.0
        
        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


# Test the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

test_model = DQNNetwork(n_actions=6, n_frames=4).to(device)
test_input = torch.zeros(1, 4, 84, 84, dtype=torch.uint8).to(device)
test_output = test_model(test_input)
print(f"✅ Model working! Input shape: {test_input.shape}, Output shape: {test_output.shape}")

Using device: cuda
✅ Model working! Input shape: torch.Size([1, 4, 84, 84]), Output shape: torch.Size([1, 6])


In [4]:
# Cell 4: Replay Buffer (Memory-Optimized)

Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Memory-optimized experience replay buffer for DQN.
    Uses numpy arrays for efficient storage instead of deque.
    
    The capacity parameter acts as max_memory_length - when the buffer is full,
    new entries automatically overwrite the oldest entries (circular buffer).
    This prevents memory from growing unbounded.
    """
    def __init__(self, capacity=250000):  # capacity = max_memory_length (reduced from 1M to 250k to prevent MemoryError)
        self.capacity = capacity
        self.size = 0
        self.ptr = 0
        
        # Pre-allocate numpy arrays for efficient storage
        # State shape: (4, 84, 84) uint8
        self.states = np.zeros((capacity, 4, 84, 84), dtype=np.uint8)
        self.next_states = np.zeros((capacity, 4, 84, 84), dtype=np.uint8)
        self.actions = np.zeros(capacity, dtype=np.int64)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=np.float32)
    
    def push(self, state, action, reward, next_state, done):
        """
        Add experience to buffer.
        Automatically overwrites oldest entry when buffer is full (circular buffer).
        This prevents memory from growing unbounded.
        """
        self.states[self.ptr] = state
        self.next_states[self.ptr] = next_state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.dones[self.ptr] = float(done)
        
        # Circular buffer: wrap around and overwrite oldest entry when full
        self.ptr = (self.ptr + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)
    
    def sample(self, batch_size, device):
        """Sample batch of experiences."""
        indices = np.random.randint(0, self.size, size=batch_size)
        
        # Use pre-allocated arrays for efficiency
        states = torch.tensor(self.states[indices], dtype=torch.uint8, device=device)
        actions = torch.tensor(self.actions[indices], dtype=torch.long, device=device)
        rewards = torch.tensor(self.rewards[indices], dtype=torch.float32, device=device)
        next_states = torch.tensor(self.next_states[indices], dtype=torch.uint8, device=device)
        dones = torch.tensor(self.dones[indices], dtype=torch.float32, device=device)
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return self.size


print("✅ Replay Buffer ready (Memory-Optimized)")

✅ Replay Buffer ready (Memory-Optimized)


In [5]:
# Cell 5: DQN Agent

class DQNAgent:
    """DQN Agent with experience replay and target network."""
    def __init__(
        self,
        n_actions,
        device,
        lr=6.25e-5,
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.1,
        epsilon_decay_steps=1000000,
        target_update_freq=10000,
        buffer_size=1000000,
        batch_size=32
    ):
        self.n_actions = n_actions
        self.device = device
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_steps = epsilon_decay_steps
        self.target_update_freq = target_update_freq
        self.batch_size = batch_size
        self.steps = 0
        
        # Networks
        self.q_network = DQNNetwork(n_actions).to(device)
        self.target_network = DQNNetwork(n_actions).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr, eps=1.5e-4)
        
        # Replay buffer (reduced size to prevent MemoryError)
        self.replay_buffer = ReplayBuffer(buffer_size)
    
    def get_epsilon(self):
        """
        Get current epsilon value using linear decay.
        
        This is the STANDARD approach used in DQN papers:
        - Linear decay from epsilon_start to epsilon_end over epsilon_decay_steps
        - Decay is based on training steps (not memory size)
        - Formula: epsilon = start - (start - end) * (steps / decay_steps)
        
        Alternative approaches (NOT recommended):
        1. Memory-based decay: epsilon -= (start-end)/MEMORY_SIZE
           - Problem: Decay depends on buffer fill rate, not training progress
           - Can decay too fast if buffer fills quickly
        
        2. Per-step decrement: epsilon -= interval / frames
           - Problem: Requires storing epsilon as instance variable
           - Less flexible than computed decay
        
        Current approach is optimal: deterministic, step-based, and matches DQN literature.
        """
        if self.steps < self.epsilon_decay_steps:
            # Linear decay: goes from start to end over decay_steps
            epsilon = self.epsilon_start - (self.epsilon_start - self.epsilon_end) * (self.steps / self.epsilon_decay_steps)
            return max(epsilon, self.epsilon_end)
        return self.epsilon_end
    
    def select_action(self, state, training=True):
        """Select action using epsilon-greedy policy."""
        if training and np.random.random() < self.get_epsilon():
            return np.random.randint(self.n_actions)
        
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.uint8, device=self.device).unsqueeze(0)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(1).item()
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay buffer."""
        self.replay_buffer.push(state, action, reward, next_state, done)
        self.steps += 1
    
    def train_step(self):
        """Perform one training step."""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Sample batch
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(
            self.batch_size, self.device
        )
        
        # Compute Q-values
        q_values = self.q_network(states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Compute target Q-values using target network
        with torch.no_grad():
            next_q_values = self.target_network(next_states)
            next_q_value = next_q_values.max(1)[0]
            target_q_value = rewards + (1 - dones) * self.gamma * next_q_value
        
        # Compute loss and optimize
        loss = F.smooth_l1_loss(q_value, target_q_value)

        
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10)
        self.optimizer.step()
        
        # Update target network periodically
        if self.steps % self.target_update_freq == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()
    
    def save(self, filepath):
        """Save model."""
        torch.save({
            'q_network': self.q_network.state_dict(),
            'target_network': self.target_network.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'steps': self.steps,
        }, filepath)
    
    def load(self, filepath):
        """Load model."""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.q_network.load_state_dict(checkpoint['q_network'])
        self.target_network.load_state_dict(checkpoint['target_network'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.steps = checkpoint['steps']


print("✅ DQN Agent ready")

✅ DQN Agent ready


In [6]:
# Cell 6: Training Loop

# Hyperparameters
TOTAL_STEPS = 5000000
LEARN_START = 12500
TRAIN_FREQ = 4
SAVE_FREQ = 200000
EVAL_FREQ = 50000

# Create environment
env = make_atari_env('ALE/Pong-v5')
n_actions = env.action_space.n

# Create agent
# Reduced buffer_size from 1M to 250k to prevent MemoryError
# 250k is still sufficient for good performance and uses ~14GB instead of ~56GB
agent = DQNAgent(
    n_actions=n_actions,
    device=device,
    lr=6.25e-5,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.1,
    epsilon_decay_steps=250000,
    target_update_freq=10000,
    buffer_size=250000,  # Reduced from 1M to prevent MemoryError
    batch_size=32
)

# Training statistics
# Use deque with maxlen to automatically pop old history and prevent memory growth
# Keep last 10000 episodes (should be more than enough for statistics)
# This prevents unbounded memory growth during long training sessions
MAX_EPISODE_HISTORY = 10000
episode_rewards = deque(maxlen=MAX_EPISODE_HISTORY)
episode_lengths = deque(maxlen=MAX_EPISODE_HISTORY)
current_reward = 0
current_length = 0

# Memory safety summary:
# ✅ Replay Buffer: Circular buffer with capacity=250k (~14GB fixed)
# ✅ Episode Stats: deque with maxlen=10k (~80KB fixed)
# ✅ FrameStack: deque with maxlen=4 (tiny, ~1KB)
# ✅ All data structures are bounded and auto-pop old entries

# Create models directory
os.makedirs('models', exist_ok=True)

print(f"🚀 Starting training for {TOTAL_STEPS:,} steps")
print(f"   Environment: Pong")
print(f"   Actions: {n_actions}")
print(f"   Device: {device}")
print(f"\n📊 Memory Safety Measures:")
print(f"   ✅ Replay Buffer: {agent.replay_buffer.capacity:,} capacity (circular, bounded)")
print(f"   ✅ Episode History: {MAX_EPISODE_HISTORY:,} max episodes (auto-pop)")
print(f"   ✅ All data structures are memory-bounded")
print(f"\n🎯 Epsilon Decay Strategy:")
print(f"   ✅ Linear decay: {agent.epsilon_start:.2f} → {agent.epsilon_end:.2f} over {agent.epsilon_decay_steps:,} steps")
print(f"   ✅ Step-based (not memory-based) - matches DQN literature")
print(f"   ✅ Deterministic and predictable decay schedule")

# Initialize state
state, info = env.reset()

# Training loop
pbar = tqdm(range(TOTAL_STEPS), desc="Training")

for step in pbar:
    # Select action
    action = agent.select_action(state, training=True)
    
    # Take step
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    # Store transition
    agent.store_transition(state, action, reward, next_state, done)
    
    # Update statistics
    current_reward += reward
    current_length += 1
    
    # Train
    if agent.steps >= LEARN_START and agent.steps % TRAIN_FREQ == 0:
        loss = agent.train_step()
        if loss is not None:
            # Convert deque to list for numpy operations
            avg_reward = np.mean(list(episode_rewards)[-10:]) if len(episode_rewards) >= 10 else 0
            pbar.set_postfix({
                'epsilon': f'{agent.get_epsilon():.3f}',
                'loss': f'{loss:.4f}',
                'avg_reward': f'{avg_reward:.1f}' if len(episode_rewards) >= 10 else 'N/A'
            })
    
    # Handle episode end
    if done:
        episode_rewards.append(current_reward)
        episode_lengths.append(current_length)
        current_reward = 0
        current_length = 0
        state, info = env.reset()
    else:
        state = next_state
    
    # Save model
    if step > 0 and step % SAVE_FREQ == 0:
        model_path = f'models/dqn_pong_{step//1000}k.pth'
        agent.save(model_path)
        print(f"\n💾 Model saved: {model_path}")
    
    # Evaluate
    if step > 0 and step % EVAL_FREQ == 0 and len(episode_rewards) >= 10:
        # Convert deque to list for numpy operations
        rewards_list = list(episode_rewards)
        lengths_list = list(episode_lengths)
        avg_reward = np.mean(rewards_list[-10:])
        avg_length = np.mean(lengths_list[-10:])
        print(f"\n📊 Step {step:,}:")
        print(f"   Avg reward (last 10): {avg_reward:.2f}")
        print(f"   Avg length: {avg_length:.1f}")
        print(f"   Epsilon: {agent.get_epsilon():.3f}")
        
        if avg_reward > 0:
            print("   🎉 Agent is winning!")
        elif avg_reward > -10:
            print("   📈 Agent is learning!")
        else:
            print("   ⏳ Still exploring...")

# Final save
agent.save('models/dqn_pong_final.pth')
env.close()
pbar.close()

print(f"\n✅ Training complete!")
print(f"Total episodes recorded: {len(episode_rewards)}")
if len(episode_rewards) > 0:
    # Convert deque to list for numpy operations
    rewards_list = list(episode_rewards)
    print(f"Final avg reward (last 100): {np.mean(rewards_list[-100:]):.2f}")
    print(f"Best episode: {max(rewards_list):.2f}")

🚀 Starting training for 5,000,000 steps
   Environment: Pong
   Actions: 6
   Device: cuda

📊 Memory Safety Measures:
   ✅ Replay Buffer: 250,000 capacity (circular, bounded)
   ✅ Episode History: 10,000 max episodes (auto-pop)
   ✅ All data structures are memory-bounded

🎯 Epsilon Decay Strategy:
   ✅ Linear decay: 1.00 → 0.10 over 250,000 steps
   ✅ Step-based (not memory-based) - matches DQN literature
   ✅ Deterministic and predictable decay schedule


Training:   1%|          | 50080/5000000 [00:50<1:43:29, 797.16it/s, epsilon=0.819, loss=0.0160, avg_reward=-20.7] 


📊 Step 50,000:
   Avg reward (last 10): -20.70
   Avg length: 880.0
   Epsilon: 0.820
   ⏳ Still exploring...


Training:   2%|▏         | 100114/5000000 [01:53<1:44:29, 781.52it/s, epsilon=0.639, loss=0.0050, avg_reward=-20.7]


📊 Step 100,000:
   Avg reward (last 10): -20.70
   Avg length: 932.9
   Epsilon: 0.640
   ⏳ Still exploring...


...


📊 Step 3,700,000:
   Avg reward (last 10): 12.20
   Avg length: 2426.0
   Epsilon: 0.100
   🎉 Agent is winning!


Training:  75%|███████▌  | 3750084/5000000 [1:38:38<34:59, 595.27it/s, epsilon=0.100, loss=0.0015, avg_reward=13.2]


📊 Step 3,750,000:
   Avg reward (last 10): 13.20
   Avg length: 2363.3
   Epsilon: 0.100
   🎉 Agent is winning!


Training:  75%|███████▌  | 3760919/5000000 [1:38:56<32:35, 633.57it/s, epsilon=0.100, loss=0.0017, avg_reward=14.9]


KeyboardInterrupt: 